# RFM (DASK)

In [1]:
import dask.dataframe as dd
from datetime import datetime
import numpy as np
import pandas as pd

Leemos el parquet.

In [5]:
df= dd.read_csv('2019-Oct-Nov-transformed.csv')
df

,event_time,event_type,product_id,category_code,brand,price,user_id
npartitions=1,,,,,,,
,object,object,int64,object,object,float64,int64
,...,...,...,...,...,...,...


In [53]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 7 entries, event_time to user_id
dtypes: object(4), float64(1), int64(2)

### RECENCY

In [6]:
window= np.datetime64('2019-12-01','ns')
window = window.astype("datetime64[D]")

In [7]:
df_recency = df.groupby('user_id').agg({'event_time': ['max']}).reset_index()
df_recency.columns = df_recency.columns.droplevel(1)
df_recency['event_time']= dd.to_datetime(df_recency['event_time'],format='%Y-%m-%d')
df_recency

,user_id,event_time
npartitions=1,,
,int64,datetime64[ns]
,...,...


In [8]:
df_recency['Recency'] = df_recency['event_time'].apply(lambda x: (window - x)).dt.days
df_recency

/opt/conda/lib/python3.10/site-packages/dask/dataframe/core.py:4134: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('event_time', 'timedelta64[ns]'))

  warnings.warn(meta_warning(meta))


,user_id,event_time,Recency
npartitions=1,,,
,int64,datetime64[ns],int64
,...,...,...


In [9]:
df_recency = df_recency[['user_id','Recency']]
df_recency.compute()

,user_id,Recency
0,543272936,2
1,551377651,6
2,514591159,61
3,555332717,35
4,524601178,14
...,...,...
697465,566369707,1
697466,579966375,1
697467,579967058,1
697468,579876821,1


### FREQUENCY

In [58]:
#df_frequency = df.groupby('user_id').event_time.count()

df_frequency = df.drop_duplicates().groupby('user_id').agg({'event_time': ['count']}).reset_index()
df_frequency.columns = df_frequency.columns.droplevel(1)
df_frequency

,user_id,event_time
npartitions=1,,
,int64,int64
,...,...


In [59]:
df_frequency= df_frequency.rename(columns={'event_time':'Frequency'})
df_frequency.compute()

,user_id,Frequency
0,138340325,1
1,225644257,1
2,253299396,1
3,256164170,1
4,264649825,2
...,...,...
697465,579962677,1
697466,579963523,1
697467,579964521,1
697468,579966375,1


### MONETARY

In [60]:
df_monetary = df.groupby('user_id').agg({'price':['sum']}).reset_index()
df_monetary.columns = df_monetary.columns.droplevel(1)

df_monetary

,user_id,price
npartitions=1,,
,int64,float64
,...,...


In [61]:
df_monetary=df_monetary.rename(columns={'price':'Monetary'})
df_monetary.compute()

,user_id,Monetary
0,138340325,93.50
1,225644257,40.91
2,253299396,246.85
3,256164170,113.23
4,264649825,1240.04
...,...,...
697465,579962677,1634.38
697466,579963523,457.91
697467,579964521,334.60
697468,579966375,124.11


### MERGE LOS TRES DF

In [62]:
df_rfm = df_recency.merge(df_frequency, how='inner')
df_rfm

,user_id,Recency,Frequency
npartitions=1,,,
,int64,int64,int64
,...,...,...


In [63]:
df_rfm = df_rfm.merge(df_monetary, how = 'inner')
df_rfm

,user_id,Recency,Frequency,Monetary
npartitions=1,,,,
,int64,int64,int64,float64
,...,...,...,...


### Eliminamos outliers utilizando el percentil 98.

In [64]:
#Chequeamos máximos y mínimos en Recency (no puede ser menor a 1 ni mayor a 61)
print(df_rfm['Recency'].min())
print(df_rfm['Recency'].max())

dd.Scalar<series-..., dtype=int32>
dd.Scalar<series-..., dtype=int32>


In [65]:
#Chequeamos máximos y mínimos en Frequency
print(df_rfm['Frequency'].min())
print(df_rfm['Frequency'].max())

dd.Scalar<series-..., dtype=int32>
dd.Scalar<series-..., dtype=int32>


In [66]:
#Chequeamos máximos y mínimos en Monetary
print(df_rfm['Monetary'].min())
print(df_rfm['Monetary'].max())

dd.Scalar<series-..., dtype=float64>
dd.Scalar<series-..., dtype=float64>


In [67]:
#Para Frequency, calculamos el percentil 98 (q98) y nos quedamos con los valores menores a él.

p98 = df_rfm['Frequency'].quantile(0.98)
print('Percentil 98: ' + str(p98))

df_rfm = df_rfm[df_rfm.Frequency <= p98]
df_rfm

Percentil 98: dd.Scalar<quantil..., dtype=float64>


,user_id,Recency,Frequency,Monetary
npartitions=1,,,,
,int64,int64,int64,float64
,...,...,...,...


In [68]:
#Para Monetary, calculamos el percentil 2 y el percentil 98 (p2 y p98) y nos quedamos con el intervalo entre estos dos valores.
p2 = df_rfm['Monetary'].quantile(0.02)
p98 = df_rfm['Monetary'].quantile(0.98)

print('Percentil 2: ' + str(p2))
print('Percentil 98: ' + str(p98))

df_rfm = df_rfm[df_rfm.Monetary > p2]
df_rfm = df_rfm[df_rfm.Monetary < p98]
df_rfm

Percentil 2: dd.Scalar<quantil..., dtype=float64>
Percentil 98: dd.Scalar<quantil..., dtype=float64>


,user_id,Recency,Frequency,Monetary
npartitions=1,,,,
,int64,int64,int64,float64
,...,...,...,...


In [69]:
df_rfm.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, user_id to Monetary
dtypes: float64(1), int64(3)

### Calculamos quintiles

In [70]:
rfm = df_rfm.compute()

R_condition = [(rfm['Recency'] < rfm['Recency'].quantile(0.2)),
            ((rfm['Recency'] >= rfm['Recency'].quantile(0.2)) & (rfm['Recency'] < rfm['Recency'].quantile(0.4))),
            ((rfm['Recency'] >= rfm['Recency'].quantile(0.4)) & (rfm['Recency'] < rfm['Recency'].quantile(0.6))),
            ((rfm['Recency'] >= rfm['Recency'].quantile(0.6)) & (rfm['Recency'] < rfm['Recency'].quantile(0.8))),
            (rfm['Recency'] >= rfm['Recency'].quantile(0.8))]

option = [5,4,3,2,1]
rfm['R'] = np.select(R_condition, option,default=5)
rfm

,user_id,Recency,Frequency,Monetary,R
0,138340325,20,1,93.50,3
1,225644257,19,1,40.91,3
2,253299396,25,1,246.85,3
3,256164170,11,1,113.23,4
4,264649825,56,2,1240.04,1
...,...,...,...,...,...
697465,579962677,1,1,1634.38,5
697466,579963523,1,1,457.91,5
697467,579964521,1,1,334.60,5
697468,579966375,1,1,124.11,5


In [71]:
F_condition = [(rfm['Frequency'] == 1) | (rfm['Frequency'] == 2),
            (rfm['Frequency'] == 3) | (rfm['Frequency'] == 4),
            (rfm['Frequency'] == 5) | (rfm['Frequency'] == 6),
            (rfm['Frequency'] == 7) | (rfm['Frequency'] == 8),
            (rfm['Frequency'] >= 9)]

option = [1,2,3,4,5]
rfm['F'] = np.select(F_condition, option,default=5)
rfm

,user_id,Recency,Frequency,Monetary,R,F
0,138340325,20,1,93.50,3,1
1,225644257,19,1,40.91,3,1
2,253299396,25,1,246.85,3,1
3,256164170,11,1,113.23,4,1
4,264649825,56,2,1240.04,1,1
...,...,...,...,...,...,...
697465,579962677,1,1,1634.38,5,1
697466,579963523,1,1,457.91,5,1
697467,579964521,1,1,334.60,5,1
697468,579966375,1,1,124.11,5,1


In [72]:
M_condition = [(rfm['Monetary'] < rfm['Monetary'].quantile(0.2)),
            ((rfm['Monetary'] >= rfm['Monetary'].quantile(0.2)) & (rfm['Monetary'] < rfm['Monetary'].quantile(0.4))),
            ((rfm['Monetary'] >= rfm['Monetary'].quantile(0.4)) & (rfm['Monetary'] < rfm['Monetary'].quantile(0.6))),
            ((rfm['Monetary'] >= rfm['Monetary'].quantile(0.6)) & (rfm['Monetary'] < rfm['Monetary'].quantile(0.8))),
            (rfm['Monetary'] >= rfm['Monetary'].quantile(0.8))]

option = [1,2,3,4,5]
rfm['M'] = np.select(M_condition, option,default=5)
rfm

,user_id,Recency,Frequency,Monetary,R,F,M
0,138340325,20,1,93.50,3,1,1
1,225644257,19,1,40.91,3,1,1
2,253299396,25,1,246.85,3,1,3
3,256164170,11,1,113.23,4,1,2
4,264649825,56,2,1240.04,1,1,5
...,...,...,...,...,...,...,...
697465,579962677,1,1,1634.38,5,1,5
697466,579963523,1,1,457.91,5,1,4
697467,579964521,1,1,334.60,5,1,3
697468,579966375,1,1,124.11,5,1,2


In [73]:
rfm["RFM_SCORE"] = (rfm['R'].astype(str) + 
                    rfm['F'].astype(str) +
                    rfm['M'].astype(str))
rfm

,user_id,Recency,Frequency,Monetary,R,F,M,RFM_SCORE
0,138340325,20,1,93.50,3,1,1,311
1,225644257,19,1,40.91,3,1,1,311
2,253299396,25,1,246.85,3,1,3,313
3,256164170,11,1,113.23,4,1,2,412
4,264649825,56,2,1240.04,1,1,5,115
...,...,...,...,...,...,...,...,...
697465,579962677,1,1,1634.38,5,1,5,515
697466,579963523,1,1,457.91,5,1,4,514
697467,579964521,1,1,334.60,5,1,3,513
697468,579966375,1,1,124.11,5,1,2,512


### A partir del RFM_SCORE, mapeamos los segmentos.

In [76]:
seg_map = dd.read_csv('segmentos_rfm.csv', usecols=['RFM_SCORE','segment']).compute()
seg_map['RFM_SCORE']= seg_map.RFM_SCORE.astype(str)
seg_map.head()

,RFM_SCORE,segment
0,111,Lost
1,112,Lost
2,113,Lost
3,114,At Risk
4,115,At Risk


In [77]:
rfm = rfm.merge(seg_map, on='RFM_SCORE')

rfm

,user_id,Recency,Frequency,Monetary,R,F,M,RFM_SCORE,segment
0,138340325,20,1,93.50,3,1,1,311,About To Sleep
1,225644257,19,1,40.91,3,1,1,311,About To Sleep
2,282274853,23,1,79.15,3,1,1,311,About To Sleep
3,310402868,20,1,77.22,3,1,1,311,About To Sleep
4,314102803,19,1,61.01,3,1,1,311,About To Sleep
...,...,...,...,...,...,...,...,...,...
658021,515913454,46,8,127.96,1,4,2,142,At Risk
658022,531241639,47,7,158.17,1,4,2,142,At Risk
658023,554615276,55,7,138.70,1,4,2,142,At Risk
658024,518195785,47,10,156.99,1,5,2,152,At Risk
